# Final Jupyter Notebook  
## Module 1 Project  
#### David Stearns  
#### Chris Fiorentine

In [1]:
# Installing of the possible packages needed\n",
!pip install requests
import requests
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import config
% matplotlib inline
sns.set(style='darkgrid')

UsageError: Line magic function `%` not found.


# API Calls

In [ ]:
# Creating a list of 500 urls for the 500 pages of data I need
# For each number in the range of [1-501) add it where 'i' is in the url below
# Using F String Interpolation to append private keys
url1 = (\"http://api.themoviedb.org/3/discover/movie?api_key={api}{synt}\").format(api=config.api, synt='&sort_by=popularity.desc&page=')
url_list = []
num_list = list(range(1,501))
url_list2 = [str(num) for num in num_list]
for i in url_list2:
    url_list.append(url1 + i)
# url_list

In [ ]:
def cleanAPI(list, key):
    movies = []
    # Creating a new list for store all of my url get requests\n",
    for url in list:
        movies.append(requests.get(url))
        
    # Creating a new list to store all of my response-object-JSON's from the get requests
    movies_json = []
    for movie in movies:
        movies_json.append(movie.json())
    # New list to store all of the 'results' for each movie
    movies_results = []
    for item in movies_json:
    # Using concatenation, not appending
        movies_results = movies_results + item['key']
    return movie_results

In [ ]:
# Confirming the code added 10,000 separate dictionaries to this list
# Not 500 separate lists (1 list per page)
len(movies_results)
# Success

In [ ]:
# Creating dataframe from data
movies_df = pd.DataFrame.from_dict(movies_results)

In [ ]:
# Sorting the list of 10k movies by popularity
# Permanently changing the data structure via 'inplace=True'
movies_df.sort_values(by='popularity', ascending=False, inplace=True)
movies_df.set_index('title', inplace=True)

In [ ]:
# Check
movies_df.head(3)

In [ ]:
# Saving a dataframe as a csv containing info from all of the movies via Discover API on TMDB
# movies_df[['popularity']].to_csv(r'/Users/davestearns/Documents/FlatIron/Mod1 Project/mod1_project-master\\popularity_df.csv')

# Retrieving IMDB ID's

In [ ]:
# Importing the big list of links made from web scraping
big_links = pd.read_csv('big_links.csv')
# Reducing the DF to include just the links as a column
# Originally had a column named 'Unnamed: 0' was a second index column - unnecessary
big_links = big_links[['0']]

In [ ]:
# Changing column name
big_links.rename(columns = {'0':'links'}, inplace=True)
imdb_id = []
# Taking ID's from index 10-4989
for i in big_links['links']:
    imdb_id.append(i[37:46])
imbd_id = imdb_id[10:5000]
# imdb_id
# Making the ID's a dataframe
imdb_df = pd.DataFrame(imbd_id)
# imdb_df

# Webscraping Data


In [ ]:
# Importing a CSV into Pandas
# File contains bs4 object strings
soups = pd.read_csv('soups_file.csv')
# Reducing the dataframe
soups = soups[['1']]

In [3]:
# Adding column to the soups DF to include the IMDB ID
soups['id'] = imdb_df
# Making a dataframe of just the soup object strings
clean = soups.iloc[0: , 0:1]
# clean

### Pulling the 'Worldwide Revenue' data from BoxOfficeMojo

In [ ]:
# Creating function to clean up the process
# In this case, str_list = clean
# Index will refer to which index we want to use to pull $ info from list of 'money'
# Index = int
revs_list = []
def wwRevs(str_list, index):
    bs4 = str_list
    win1 = []
    for bs in bs4:
        win1.append(BeautifulSoup(bs, 'html.parser'))
    # Loading in bs4 objects from the dataframe to new list
    # Index 2
    win2 = []
    for won in win1:
        try:
            win2.append(won.findAll('span', class_= 'money')[index].text.strip('$'))
        except IndexError:
            type(win2[0])
    # Cleaning numbers
    # Transforming all str --> int
    # Removing commas in numbers
    for num in win2:
        revs_list.append(int(num.replace(',', '')))
    return revs_list[0]
# Returns one value from this list as a check to make sure it worked

In [ ]:
def descStats(revs_list):
    # Turn list into PD df
    wwRev_df = pd.DataFrame(revs_list)
    return (
            wwRev_df.mean(),
            wwRev_df.median(),
            wwRev_df.mode(),
            wwRev_df.describe(),
            wwRev_df.sum()
    )

In [ ]:
# index2 = wwRevs(clean['1'], 2)

In [ ]:
revs_df = pd.DataFrame(revs_list[0:4990])
revs_df

In [ ]:
# index2_stats = descStats(revs_list)

In [ ]:
# Renaming the column to something more descriptive
revs_df.rename(columns = {0:'ww_rev'}, inplace=True)

In [ ]:
# Adding a column for the IMDB ID
# Will be using this to merge the dataframes later
revs_df['id'] = imdb_df[0]

In [ ]:
# Creating a list of IMDB ID's to add in to the API url to find data
id_list = imdb_df[0].to_list()
id_list
movie_list = []
for i in id_list:
    movie_list.append(str(f"https://api.themoviedb.org/3/find/{i}?api_key=41f8edb29d800a88a356853208f114f0&language=en-US&external_source=imdb_id\"))

Good candidate for `cleanAPI()`  
*The cell below may take some time depending on the size of your data.*

In [ ]:
movies8 = []
for movie in movie_list:
    movies8.append(requests.get(movie))
# Creating a new list to store all of my response-object-JSON's from the get requests
movies_json8 = []
for movie in movies8:
    movies_json8.append(movie.json())
# New list to store all of the 'results' for each movie
movies_results8 = []
for item in movies_json8:
# Using concatenation, not appending
    movies_results8 = movies_results8 + item['movie_results']
# Creating PD df from results
movies_df8 = pd.DataFrame.from_dict(movies_results8)

In [ ]:
# Dropping the current 'id' column and replacing it with a more accurate one
# New column has the IMDB ID's
movies_df8.drop(['id'], axis = 1, inplace=True)
movies_df8['id'] = id_list[0:4982]

Will need to apply the ID column to this PD DF to merge the wwRev and the API table

In [ ]:
# Merging the new PD df with the dataframe containing revenue data
# Merging on 'id' column
movies_test = movies_df8
movies_final = movies_test.merge(revs_df, on='id')
movies_final.set_index('title', inplace=True)

In [ ]:
movies_final.sort_values('ww_rev', ascending=False, inplace=True)

In [ ]:
# movies_final.to_csv(r'/Users/davestearns/Documents\\movies_final9000.csv')

In [ ]:
# Making a more simple df
movies_finale = movies_final[['id', 'popularity', 'ww_rev']]

In [ ]:
movies_finale = movies_finale.drop_duplicates()

In [ ]:
# movies_finale.to_csv(r'/Users/davestearns/Documents\\movies_finale.csv')

In [ ]:
# To reduce the size of the data and to reduce low value bias
graph_df = movies_final[movies_final['ww_rev'] > 5000000]
graph_df

In [ ]:
sns.set_style('whitegrid')
populs = graph_df['popularity']
revs = graph_df['ww_rev']
graph2 = sns.scatterplot(x = revs, y = populs, hue=graph_df['popularity'], data = graph_df)
graph2.set(xlabel='Revenue', ylabel='Popularity')
graph2.set_xticklabels(['$0M', '$50M', '$100M', '$150M', '$200M', '$250M', '$300M', '$350M', '$400M'])
plt.title('Comparing Revenue to Popularity')
plt.xlim(50000000, 400000000)

Correlate the Revenue to Popularity

In [ ]:
graph_df['popularity'].corr(graph_df['ww_rev'])

In [ ]:
# Making a new dataframe to further reduce the influence on outliers
# There weren't many movies above a popularity of 170, so to reduce outlier influence action was taken
# Values below 5 were making the visualization hard to see - they were omitted
# Vote Counts below 1000 were omitted to combat outlier influence
movies_final2 = movies_final[(movies_final['popularity'] < 170) & (movies_final['popularity'] > 5) & (movies_final['vote_count'] > 1000)]
pop = movies_final2['popularity']
votes = movies_final2['vote_count']
graph1 = sns.scatterplot(x = votes, y = pop, hue = movies_final2['vote_average'])
graph1.set(xlabel='Vote Count', ylabel='Popularity')
plt.title('Comparing Popularity to Vote Count')
popvote_corr = movies_final2['vote_count'].corr(movies_final2['popularity'])
popvote_corr

In [ ]:
# Seeing what movie had the highest popularity, and looking at its revenue
# EDA purposes
movies_final2[movies_final2['popularity'] > 40]

In [ ]:
# Reusing previous dataframe for this visualization
# Sorting it based on revenue, highest first
# graph_df.sort_values('ww_rev', ascending=False, inplace=True)
# Making a list of the first 10 names
names = movies_final.index[0:10].to_list()
top10_grossing_movies = sns.barplot(x=names, y=movies_final['ww_rev'][0:10])
top10_grossing_movies.set_xticklabels(
    top10_grossing_movies.get_xticklabels(),
    rotation = 45, horizontalalignment = 'right')
plt.ylim(20000000, 3000000000)
y_labels = ['$0.5B', '$1.0B', '$1.5B', '$2.0B', '$2.5B', '$3.0B']
top10_grossing_movies.set_yticklabels(y_labels)
plt.title('Top 10 Grossing Movies (Worldwide Revenue)')
top10_grossing_movies.set(xlabel='Top 10 Grossing Movies Worldwide', ylabel='Revenue')

### Calculating Profit Velocity

In [ ]:
# Needed to convert datatypes to and from datetime objects
# Needed for date analysis
from datetime import datetime

In [ ]:
vlist = movies_final['release_date'].to_list()
movies_final['vlist'] = vlist
movies_final.dropna(subset=['vlist'], inplace=True)
vlist2 = movies_final['vlist'].to_list()
# movies_final

In [ ]:
vlist3 = []
for v in vlist2:
    vlist3.append(str(v))
movies_final['vlist3'] = vlist3
movies_final.dropna(inplace=True)
vlist4 = movies_final['vlist3'].to_list()
# vlist3
vlistadd = []
for v in vlist4:
    vlistadd.append(datetime.strptime(v, \"%Y-%m-%d\"))

In [ ]:
movies_final['release'] = vlistadd

In [ ]:
movies_final.drop(columns = ['vlist', 'release_date'], axis = 1, inplace=True)
movies_final.drop(columns='index', inplace=True)
movies_final.drop(columns='vlist3', axis=1, inplace=True)

In [ ]:
today = '2020-03-05'
todate = datetime.strptime(today, \"%Y-%m-%d\")
movies_final['release'][0]

In [ ]:
movies_final['time_elapsed'] = todate - movies_final['release']
movies_final['time_elapsed'] = movies_final['time_elapsed'].apply(lambda x: int(x.days))
movies_final['time_elapsed']
movies_final.head()

In [ ]:
movies_final['profit_velocity'] = movies_final['ww_rev']/movies_final['time_elapsed']

In [ ]:
# Reducing the dataset to include movies within the past 25 years
velocity = movies_final[(movies_final['time_elapsed'] < 12000) & (movies_final['time_elapsed'] > 6000)]

In [ ]:
names = velocity.index[0:10].to_list()
top10_grossing_movies = sns.barplot(x=names, y=velocity['profit_velocity'][0:10])
top10_grossing_movies.set_xticklabels(
    top10_grossing_movies.get_xticklabels(),
    rotation = 45, horizontalalignment = 'right')
plt.ylim(0, 350000)
y_labels = ['$0', '$50K', '$100K', '$150K', '$200K', '$250K', '$300K']
top10_grossing_movies.set_yticklabels(y_labels)
plt.title('Top Movers of the Past Generation')
top10_grossing_movies.set(xlabel='Top 10 Movers', ylabel='Profit Velocity ($/Day)')

In [ ]:
velocity_new = velocity[velocity['ww_rev'] > 6000000]

In [ ]:
velocity_new['ww_rev'].sum()

In [ ]:
velocity_new.describe()